# Lab5 - Conditional Sequence-to-sequence VAE

In [4]:
import string
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt
from tqdm import trange
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Compute BLEU-4 score
def compute_bleu(output, reference):
    cc = SmoothingFunction()
    if len(reference) == 3:
        weights = (0.33, 0.33, 0.33)
    else:
        weights = (0.25, 0.25, 0.25, 0.25)
    return sentence_bleu([reference], output, weights=weights, smoothing_function=cc.method1)

# Compute Gaussian score
def Gaussian_score(words):
    words_list = []
    score = 0
    yourpath = 'data/train.txt'#should be your directory of train.txt
    with open(yourpath,'r') as fp:
        for line in fp:
            word = line.split(' ')
            word[3] = word[3].strip('\n')
            words_list.extend([word])
        for t in words:
            for i in words_list:
                if t == i:
                    score += 1
    return score/len(words)

In [6]:
# Define characteristic to vector dictionary for embedding
SOS_token = 0
EOS_token = 1
ch_to_ix = {ch: i+2 for i, ch in enumerate(string.ascii_lowercase)}
ix_to_ch = {i+2: ch for i, ch in enumerate(string.ascii_lowercase)}

# Define condition to dictionary
conditions = ['sp', 'tp', 'pg', 'p']
cond_to_ix = {conditions[i]: i for i in range(len(conditions))}

# Load the datasets
def load_data(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            data.append(line.rstrip().split(' '))
    return data


train_data = load_data('./data/train.txt')
train_cond = ['sp', 'tp', 'pg', 'p']

test_data = load_data('./data/test.txt')
test_cond = [
    ['sp', 'p'],
    ['sp', 'pg'],
    ['sp', 'tp'],
    ['sp', 'tp'],
    ['p', 'tp'],
    ['sp', 'pg'],
    ['p', 'sp'],
    ['pg', 'sp'],
    ['pg', 'p'],
    ['pg', 'tp'],
] # Given

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size, num_conds, cond_size):
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.cond_size = cond_size
        
        # Embedding the input
        self.word_embedding = nn.Embedding(input_size, hidden_size)
        self.cond_embedding = nn.Embedding(num_conds, cond_size)
        
        # LSTM
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        
        # Fully connected layer to generate mean and var for hidden and cell
        self.hidden_mean = nn.Linear(hidden_size, latent_size)
        self.hidden_logvar = nn.Linear(hidden_size, latent_size)
        self.cell_mean = nn.Linear(hidden_size, latent_size)
        self.cell_logvar = nn.Linear(hidden_size, latent_size)
        
    def forward(self, input, condition, hidden):
        condition = self.cond_embedding(condition).view(1, 1, -1)
        hidden = (torch.cat((hidden[0], condition), dim=2), torch.cat((hidden[1], condition), dim=2))
        output = self.word_embedding(input).view(input.size(0), 1, -1)
        output, hidden = self.lstm(output, hidden)
        
        mean = (self.hidden_mean(hidden[0]), self.cell_mean(hidden[1]))
        logvar = (self.hidden_logvar(hidden[0]), self.cell_logvar(hidden[1]))
        
        # Reparameterize
        std = (torch.exp(logvar[0]), torch.exp(logvar[1]))
        eps = (torch.rand_like(std[0]), torch.rand_like(std[1]))
        latent = (mean[0]+eps[0]*std[0], mean[1]+eps[1]*std[1])
        
        return latent, mean, logvar
        
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_size-self.cond_size, device=device),
                torch.zeros(1, 1, self.hidden_size-self.cond_size, device=device))
    
    def condition_embedding(self, condition):
        return self.cond_embedding(condition)
    
    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

In [8]:
def decode(decoder, hidden, target_tensor, teacher_forcing):
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = hidden
    
    target_length = target_tensor.size(0)
    
    outputs = []
    if teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            outputs.append(decoder_output)
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            outputs.append(decoder_output)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            
            if decoder_input.item() == EOS_token:
                break
                
    outputs = torch.cat(outputs, dim=0)
    return outputs
    

def train_pair(input_data_tensor, input_cond_tensor, target_data_tensor, target_cond_tensor,
               encoder, encoder_optimizer, decoder, decoder_optimizer, kl_weight,
               criterion, teacher_forcing_ratio=0.5, max_length=MAX_LENGTH):
    
    encoder_hidden = encoder.init_hidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_data_tensor.size(0)
    target_length = target_data_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    
    loss = 0
    
    #----------sequence to sequence part for encoder----------#
    latent, mean, logvar = encoder(input_data_tensor, input_cond_tensor, encoder_hidden)
    print(latent)
    exit(0)
    
    #----------sequence to sequence part for decoder----------#
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    decoder_cond = encoder.condition_embedding(target_cond_tensor).view(1, 1, -1)
    decoder_hidden = (torch.cat((latent[0], decoder_cond), dim=2), torch.cat((latent[1], decoder_cond), dim=2))
    output = decode(decoder, decoder_hidden, target_data_tensor, teacher_forcing=use_teacher_forcing)
    
    crossEntropy_loss = criterion(output, target_data_tensor[:output.size(0)].view(-1))
    kl_loss = torch.sum(0.5*(-logvar[0]+(mean[0]**2)+torch.exp(logvar[0])-1)) + torch.sum(0.5*(-logvar[1]+(mean[1]**2)+torch.exp(logvar[1])-1))
    (crossEntropy_loss+kl_weight*kl_loss).backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return crossEntropy_loss.item(), kl_loss.item()
    
    
def predict(input_data_tensor, input_cond_tensor, target_data_tensor, target_cond_tensor, 
            encoder, decoder, criterion, max_length=MAX_LENGTH):
    
    encoder_hidden = encoder.init_hidden()
    input_data_length = input_data_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    
    with torch.no_grad():
        
        latent, mean, logvar = encoder(input_data_tensor, input_cond_tensor, encoder_hidden)
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_cond = encoder.cond_embedding(target_cond_tensor).view(1, 1, -1)
        decoder_hidden = (torch.cat((latent[0], decoder_cond), dim=2), torch.cat((latent[1], decoder_cond), dim=2))
        
        crossEntropy_loss = 0
        kl_loss = torch.sum(0.5*(-logvar[0]+(mean[0]**2)+torch.exp(logvar[0])-1)) + torch.sum(0.5*(-logvar[1]+(mean[1]**2)+torch.exp(logvar[1])-1))
        
        pred = ''
        for di in range(target_data_tensor.size(0)):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            
            crossEntropy_loss += criterion(decoder_output, target_data_tensor[di])
            if decoder_input == EOS_token:
                break
            
            pred += ix_to_ch[decoder_input.item()]
    
    return pred, crossEntropy_loss, kl_loss
    

def train_data_combination(n=4):
    for i in range(n):
        for j in range(i+1, n):
            yield (i, j)
            yield (j, i)
            

def get_kl_anealing_func(method, slope=0.0002, max_val=1, period=None):
    if method == 'monotonic':
        return lambda iteration: min(iteration*slope, max_val)
    elif method == 'cyclical' and period:
        return lambda iteration: min((iteration%period)*slope, max_val)
    else:
        return None
    

def train(encoder, decoder, train_dataset, train_condition, test_dataset, test_condition,
          kl_anealing_func, teacher_forcing_ratio, num_epochs=20, learning_rate=1e-02):
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.CrossEntropyLoss()
    
    train_result_list = {'cross_entropy': [], 'kl': []}
    test_result_list = {'cross_entropy': [], 'kl': [], 'score': []}
    iteration = 0
    for epoch in range(num_epochs):

        # Train
        encoder.train()
        decoder.train()
        
        shuffled_idx = np.random.permutation(len(train_dataset))
        train_loss = {'cross_entropy': 0, 'kl': 0}
        for i in trange(len(train_dataset)):
            data = train_dataset[shuffled_idx[i]]
            cond = train_condition
            for a, b in train_data_combination(4):
                input_data_tensor = torch.tensor([ch_to_ix[ch] for ch in data[a]], device=device).view(-1, 1)
                input_cond_tensor = torch.tensor(cond_to_ix[train_cond[a]], device=device)
                target_data_tensor = torch.tensor([ch_to_ix[ch] for ch in data[b]]+[EOS_token], device=device).view(-1, 1)
                target_cond_tensor = torch.tensor(cond_to_ix[train_cond[b]], device=device)
                loss = train_pair(input_data_tensor, input_cond_tensor, target_data_tensor, target_cond_tensor,
                           encoder, encoder_optimizer, decoder, decoder_optimizer, kl_anealing_func(iteration),
                           criterion, teacher_forcing_ratio)
                train_loss['cross_entropy'] += loss[0]
                train_loss['kl'] += loss[1]
                
        train_loss['cross_entropy'] /= (len(train_dataset)*12)
        train_loss['kl'] /= (len(train_dataset)*12)
        train_result_list['cross_entropy'].append(train_loss['cross_entropy'])
        train_result_list['kl'].append(train_loss['kl'])
                
        # Test
        encoder.eval()
        decoder.eval()
        
        test_loss = {'cross_entropy': 0, 'kl': 0}
        test_score = 0
        for i in trange(len(test_dataset)):
            input_data_tensor = torch.tensor([ch_to_ix[ch] for ch in test_dataset[i][0]], device=device).view(-1, 1)
            input_cond_tensor = torch.tensor(cond_to_ix[test_condition[i][0]], device=device)
            target_data_tensor = torch.tensor([ch_to_ix[ch] for ch in test_dataset[i][1]], device=device).view(-1, 1)
            target_cond_tensor = torch.tensor(cond_to_ix[test_condition[i][1]], device=device)
            pred, crossEntropy_loss, kl_loss = predict(input_data_tensor, input_cond_tensor, target_data_tensor, target_cond_tensor, encoder, decoder, criterion)
            test_loss['cross_entropy'] += crossEntropy_loss
            test_loss['kl'] += kl_loss
            score = compute_bleu(pred, test_dataset[i][1])
            test_score += score
        
        test_loss['cross_entropy'] /= len(test_dataset)
        test_loss['kl'] /= len(test_dataset)
        test_result_list['cross_entropy'].append(test_loss['cross_entropy'])
        test_result_list['kl'].append(test_loss['kl'])
        test_score /= len(test_dataset)
        test_result_list['score'].append(test_score)
        
        # Print the result
        print('=====================================================')
        print('Epoch: {} / {}'.format(epoch+1, num_epochs))
        print()
        print('Train Cross Entropy Loss: {}'.format(train_loss['cross_entropy']))
        print('Train KL Loss: {}'.format(train_loss['kl']))
        print()
        print('Test Cross Entropy Loss: {}'.format(test_loss['cross_entropy']))
        print('Test KL Loss: {}'.format(test_loss['kl']))
        print('Test BLEU-4 Score: {}'.format(test_score))
        print()
        print()
         
    return encoder, decoder, train_result_list, test_result_list


def evaluate(test_dataset, test_condition, encoder, decoder, max_length=MAX_LENGTH):
    
    score = 0
    for i in range(len(test_dataset)):
        input_data_tensor = torch.tensor([ch_to_ix[ch] for ch in test_dataset[i][0]], device=device).view(-1, 1)
        input_cond_tensor = torch.tensor(cond_to_ix[test_condition[i][0]], device=device)
        target_cond_tensor = torch.tensor(cond_to_ix[test_condition[i][1]], device=device)
        pred, _, _ = predict(input_data_tensor, input_cond_tensor, target_cond_tensor, encoder, decoder, criterion)
        score += compute_bleu(pred, test_dataset[i][1])
        
        print('Input:      {}'.format(test_dataset[i][0]))
        print('Target:     {}'.format(test_dataset[i][1]))
        print('Prediction: {}'.format(pred))
        print()
    print('Average BLEU-4 Score: {}'.format(score/len(test_dataset)))

In [31]:
# Define hyperparameters
hidden_size = 256
latent_size = 128
vocab_size = 28
num_conds = 4
cond_size = 32
teacher_forcing_ratio = 0.5
learning_rate = 0.05
MAX_LENGTH = 40
num_epochs = 20

# Encoder & Decoder
encoder = EncoderRNN(vocab_size, hidden_size, latent_size, num_conds, cond_size).to(device)
decoder = DecoderRNN(latent_size+cond_size, vocab_size).to(device)

# Train
encoder, decoder, train_loss_list, test_score_list = train(encoder, decoder,
                                                           train_data, train_cond, test_data, test_cond,
                                                           kl_anealing_func=get_kl_anealing_func('cyclical', period=10000),
                                                           teacher_forcing_ratio=teacher_forcing_ratio,
                                                           num_epochs=num_epochs)

  0%|          | 1/1227 [00:00<03:30,  5.82it/s]

(tensor([[[ 0.0671,  0.4449,  0.0715,  0.6617,  0.2406,  0.4819,  0.9446,
           0.5709,  0.4930,  0.6902,  0.1864,  0.3946,  1.0738,  0.4902,
           0.0428,  0.3898,  0.8097,  0.1008,  0.1545,  0.0890,  0.7683,
           0.3007,  0.9745,  0.4894,  0.5402,  0.9945,  0.3667,  0.7451,
           0.2960,  0.9450,  0.4693,  0.6595,  0.6082,  0.6110,  0.1822,
           0.4577,  0.5219,  0.9737,  0.0662,  0.2775,  0.9171,  0.1050,
           0.6765,  0.7380,  0.7103,  0.6825,  0.3935,  0.1138,  0.7962,
          -0.1546,  0.2812,  0.0600,  0.7602,  1.0026,  0.8182,  0.5284,
           0.3488,  0.5304,  0.8471,  0.4774,  0.7818,  0.8644,  0.4314,
           0.4265,  0.4708,  0.1536,  0.3532,  0.0349,  0.6228,  0.9451,
           0.7726,  0.1605,  0.8209,  1.0051, -0.0506,  0.7538,  0.8767,
           0.6414,  0.6804,  0.3210,  0.7893,  0.2976,  0.9821,  0.7653,
           0.5381,  0.5445, -0.0137,  0.4260,  0.4036,  0.2460,  0.3226,
           0.1396,  0.7200,  0.6522,  0.0095,  0.6

  0%|          | 2/1227 [00:00<03:28,  5.86it/s]

(tensor([[[ 0.7680,  0.3549,  0.5028,  0.1547,  0.0922,  0.8049,  0.1847,
           0.5825,  0.2767,  0.3484,  0.0909,  0.2842,  0.6325,  0.1189,
           0.6327,  0.7280,  0.4446,  0.5737,  0.5235,  0.6244,  0.7580,
           0.6156,  0.0308,  0.3480,  0.6364,  0.8769,  0.9182,  0.2792,
           0.4498,  0.4578,  0.6850,  0.6476,  0.1685, -0.0429,  0.0241,
           0.4015,  0.2159,  0.6336,  0.5072,  0.2178,  0.0996,  0.8189,
           0.9469,  0.1432,  0.1960,  0.0495,  1.0482,  0.3656,  0.3159,
           0.0658,  0.5632,  0.2299,  0.1357,  0.9011,  0.6470,  0.7376,
           0.4004,  0.4832,  0.9095,  0.2230,  0.2210,  0.6531, -0.0307,
           0.6362,  0.5972,  0.8923,  0.8350,  0.9406,  0.8791,  0.0263,
           0.2602,  0.2891,  0.2747,  0.6332,  0.6878,  0.0962,  0.3933,
           0.2982,  0.1845,  0.0191,  0.5451,  0.9604,  0.4479,  0.1165,
           1.0621,  1.1266,  0.0545,  0.7332,  1.0919,  0.0024,  0.6712,
           0.1804,  0.4679,  0.1224,  0.2150,  0.7

  0%|          | 3/1227 [00:00<03:24,  5.99it/s]

...

  5%|▍         | 56/1227 [00:08<03:10,  6.15it/s]

(tensor([[[ 0.0472,  0.7635,  0.8397,  0.8038,  0.3372,  0.1390,  1.0654,
           0.2148,  0.5383, -0.0260,  0.3694,  0.7691,  0.3100,  0.3840,
           0.8904,  0.1133,  1.1325,  0.2703,  0.2430,  0.0692,  0.0813,
           0.5834,  0.2048,  0.1404,  0.5615,  1.0986,  0.4173,  0.6299,
           0.4995,  0.3380,  0.7171,  0.2806,  0.4268,  0.4920,  0.2508,
           0.8704,  0.6303,  0.3147,  0.7987,  0.0085, -0.0113,  0.2519,
           0.3583,  0.9667,  0.8462,  0.9575,  0.4292,  0.9384,  0.4457,
           0.9058,  1.0311,  0.6002, -0.0325,  0.1841,  0.9075,  0.8775,
           0.4161,  0.1816,  0.4659,  0.8138,  0.4161,  0.4678,  0.5828,
           0.7141,  0.1948,  0.9794, -0.1026,  0.7011,  0.9192,  0.7225,
           0.3654,  0.5595,  0.1461,  0.5275,  1.0229,  0.8322,  0.6192,
           0.5486,  0.7960, -0.0117,  0.3331,  0.3716,  0.1441,  0.5544,
           1.1611,  1.1040,  0.6289,  1.1191,  0.3823,  0.9648,  0.8715,
           0.4327,  0.8543,  0.6808,  0.8686,  0.0

  5%|▍         | 57/1227 [00:09<03:07,  6.23it/s]

(tensor([[[ 0.7797,  0.7136,  0.8735,  0.3865,  0.5601,  0.5458,  0.1699,
           0.1385,  0.6218,  0.2897,  0.8859,  0.5142,  0.1685,  0.8880,
           0.3183,  0.8289,  1.2757,  0.5494,  0.0283,  0.6286,  0.9168,
           0.5149,  0.3122,  0.3335,  0.0515,  0.4580,  0.3409,  0.0193,
           0.7435,  0.5731,  0.4036,  0.0843,  0.1628,  0.9220,  0.0279,
           0.6349,  0.0060,  0.4239,  0.5179,  0.9191,  0.8496,  1.0077,
           1.1453,  0.6490,  0.2922,  0.2637,  1.0449,  0.3517,  0.4411,
           0.2087,  0.5230,  0.5806,  0.7603,  0.1307,  0.1449,  0.1522,
           0.9955,  1.0965,  0.3415,  0.6035,  0.9894,  0.3294, -0.1133,
          -0.1802,  0.6061,  0.2512,  0.3650,  0.6656,  0.1416,  0.7757,
           0.2555,  0.4136,  0.7754,  0.6781,  0.4678,  0.7092,  0.6337,
           0.4153,  0.2094,  0.6247,  1.0041,  0.4786,  0.4758,  0.3844,
           0.4627,  0.4858,  0.3202,  0.7345,  0.3006,  0.7694,  0.8185,
           0.1444,  0.6026,  0.3322,  0.2459,  0.2

KeyboardInterrupt: 

In [289]:
torch.save(encoder.state_dict(), 'models/encoder_monotonic.pth')
torch.save(decoder.state_dict(), 'models/decoder_monotonic.pth')

SyntaxError: illegal expression for augmented assignment (<ipython-input-15-41ab0e08df21>, line 3)

In [7]:
# Define hyperparameters
hidden_size = 256
latent_size = 128
vocab_size = 28
num_conds = 4
cond_size = 32
teacher_forcing_ratio = 0.5
learning_rate = 0.05
MAX_LENGTH = 40
num_epochs = 20

# Encoder & Decoder
encoder = EncoderRNN(vocab_size, hidden_size, latent_size, num_conds, cond_size).to(device)
decoder = DecoderRNN(latent_size+cond_size, vocab_size).to(device)

encoder.load_state_dict(torch.load('models/encoder_cyclical.pth'))
decoder.load_state_dict(torch.load('models/decoder_cyclical.pth'))

<All keys matched successfully>

In [11]:
words_list = generate_words(encoder, decoder)

(tensor([[[-1.3554, -0.4302,  0.3166,  0.0586, -0.3410,  0.5489, -0.0815,
          -0.4609,  0.1342,  0.4785, -1.3191, -0.2082,  0.7813, -1.3948,
          -0.2952, -1.4528, -1.3076,  0.1294,  0.2870, -0.2742, -1.9832,
          -0.1390, -0.9281,  0.5579,  2.0234,  1.3393, -1.1813,  0.2878,
          -0.1760,  0.0969,  0.0297, -1.4681,  0.5995,  0.7840, -0.7095,
           0.4644, -0.0772,  0.0513, -0.2005,  0.0996, -0.1426,  0.4712,
           0.0667, -0.0848,  0.3033, -0.9096,  0.3957, -0.4060,  0.0515,
           0.0432,  0.9506, -0.8074,  0.2111, -1.0355,  0.8437,  2.4551,
          -0.4498, -0.1083, -1.5220,  1.1774, -0.9578,  1.8828,  0.9016,
          -1.1422,  0.0167,  1.3688, -0.5648,  0.0688, -2.5277,  0.8364,
          -2.0461, -0.1645,  1.2321, -1.3797,  0.6728,  1.3848,  0.2451,
          -1.2914,  0.4231,  0.0145,  0.2952,  1.4987, -0.2367, -0.5906,
          -0.2984, -0.6557, -1.1615,  0.3507, -1.0994,  0.9797, -1.9912,
           0.4515,  0.5153,  0.4650, -0.1690, -0.0

(tensor([[[-2.3550, -1.9872, -1.0729, -1.1634, -0.3705,  0.3174,  0.5894,
          -0.3252, -0.1879, -0.1672, -1.0352, -0.0373, -0.1886,  1.8623,
           0.6009, -0.5415,  0.8368, -1.4444,  2.0295, -0.0769, -1.0406,
           0.6985, -0.6286,  0.1927, -1.0598, -1.2090,  0.5353, -0.8489,
          -0.8582,  1.5506,  1.0503, -1.4356, -1.3509,  1.8993, -1.2497,
           0.2391,  1.3701, -0.0038,  1.0183, -0.7712,  0.2985,  0.0872,
           0.8486, -0.0615, -0.5202, -1.6247, -1.6553, -1.1476, -0.9570,
           1.6773,  0.4787, -0.9522,  0.8922,  1.0099, -0.6584, -0.1060,
          -0.6225,  0.5807,  0.3248, -0.3486,  0.7192, -0.3190,  0.0130,
           1.9678,  0.1860,  3.0203,  0.5907,  0.8928, -1.2865,  0.0528,
          -0.2003, -1.0825,  0.0316, -0.3443, -1.1096,  0.1107,  1.3911,
          -0.1826,  0.2013, -1.7944, -0.8927, -1.0361,  0.7301,  0.8463,
           0.9251,  0.8496, -0.8728, -0.5722,  0.5271,  0.0988,  1.4568,
           0.2953, -0.5592,  0.9553,  0.4077,  0.8

(tensor([[[ 1.7645e+00,  9.7721e-01, -2.4031e-01,  2.2376e+00, -1.4626e-01,
          -1.9992e+00, -6.3164e-01, -4.9811e-01, -5.8170e-01,  9.3369e-01,
          -6.8444e-01,  1.0617e+00,  8.0941e-02, -1.1306e+00,  6.7968e-01,
           2.8853e-01, -1.7421e+00,  1.7469e-01, -7.4127e-01, -2.0943e+00,
           1.9236e+00, -1.9765e+00,  1.3118e+00, -7.5656e-02,  1.8671e+00,
          -1.1252e-01,  2.1115e+00,  7.2884e-02,  7.1798e-01, -1.2476e+00,
          -1.0835e-01, -1.0873e-01, -5.8276e-01, -3.4436e-01,  6.8883e-02,
           1.8760e-01,  9.0691e-01, -1.7274e+00,  1.6635e-01, -9.2832e-02,
          -4.5538e-01, -4.1756e-01, -2.1702e+00,  1.8612e-01,  9.1699e-02,
          -1.2738e-01,  6.3400e-01,  1.0522e+00,  3.9063e-01, -9.0161e-01,
          -2.0238e+00, -7.6484e-01,  1.6699e+00, -1.7715e+00,  5.7862e-01,
          -1.7018e+00,  4.3745e-01, -7.0578e-01,  3.5399e-01, -1.1034e+00,
          -5.9486e-01,  1.1327e+00,  2.0601e+00, -2.5938e+00,  1.3607e+00,
           6.8587e-01, -

(tensor([[[ 0.1681, -1.2398,  0.8761, -0.3031,  0.1130,  1.0877, -0.2752,
           0.7034, -0.4465, -0.4027,  0.1365, -0.5055, -0.4281, -1.2603,
           0.6361,  1.9502,  0.6321, -0.4057, -1.0358, -1.7653,  0.4956,
          -0.5591, -0.2533, -2.7840, -1.3730,  0.9921, -0.3113,  0.0847,
           0.5533, -0.1358,  0.3246, -1.4159,  1.2251, -0.8310,  2.2451,
          -0.4034, -0.1934,  2.2379, -0.5139, -0.6489,  0.5120, -0.8119,
           0.6978,  0.5643, -0.9681,  1.5043,  0.1052, -1.1237, -0.4390,
           0.5091, -0.2627, -0.8159, -1.2607, -0.3522, -0.2672,  0.5923,
           0.6217,  1.1508,  0.7818,  0.9967, -0.0645,  1.4888,  0.5669,
          -1.5950, -2.1005,  1.2261,  1.5585,  0.6446,  0.8404, -0.6484,
          -0.7072,  1.5421,  0.4800, -0.5666, -0.7763, -0.2937, -1.5439,
           2.2993,  0.5042,  1.2806,  0.6974,  0.0688,  0.0387,  0.2810,
           2.4361, -1.0463,  0.7052,  1.4112, -0.5671, -0.9714, -0.6312,
           0.4871,  0.3862,  0.9245, -0.5786, -0.0

(tensor([[[-1.0736e+00, -8.9911e-01, -7.6442e-01,  2.4509e+00, -2.5580e-01,
          -7.6862e-02,  2.3797e+00,  7.6094e-01, -5.5715e-01,  3.4729e-01,
           4.6131e-01, -9.8068e-02,  8.4284e-01, -2.6123e-01, -6.8134e-01,
          -1.3084e+00,  9.8349e-01, -2.9111e-01,  6.9726e-02, -1.9866e+00,
          -2.7476e+00, -2.1026e-01,  2.3516e-01,  3.4338e-01, -5.1833e-01,
          -6.5271e-01,  4.8549e-01,  7.1102e-01,  1.2326e-01, -6.1937e-02,
           2.0231e-02,  7.7376e-01,  1.6573e+00,  1.0354e+00, -2.3845e-02,
           6.9863e-01,  1.3700e+00, -1.1061e+00, -1.0664e+00,  2.3430e+00,
           5.7530e-01, -1.5139e+00,  1.0287e+00,  7.6057e-02,  2.1930e-01,
           1.1695e+00,  4.4980e-01,  1.3793e+00, -1.2750e+00,  2.6100e-01,
           1.2590e+00, -2.9065e-01,  1.3169e+00, -4.6732e-01,  3.8857e-02,
           1.4383e-01,  1.0866e+00, -7.1082e-01,  8.7099e-01, -2.5276e-01,
          -4.0634e-01, -8.5508e-01,  2.5496e-04,  2.8586e-01,  1.8409e-01,
           6.2021e-01,  

...

(tensor([[[ 0.2451,  1.0155, -1.2132, -0.1183, -0.3729,  0.8197, -0.3307,
          -0.7054,  2.0540,  0.6928,  0.3093,  0.0707,  0.0229, -0.4497,
          -1.9719, -0.2459, -0.5017,  0.6213, -1.1440,  0.3519,  1.0515,
           1.2309,  1.0686,  0.2587,  0.2435,  0.7558,  1.7629, -0.6452,
           1.4713,  0.5663,  1.5311,  1.5804, -0.5570,  0.9868,  0.1609,
          -0.2523, -0.8759,  0.7641, -0.6190,  1.1508,  0.8113, -0.5843,
          -1.6737, -0.3503,  0.4408,  0.8783, -0.7723, -0.9719, -1.5527,
           1.5790,  0.4931, -0.1290,  0.7938,  0.2049, -1.2594, -0.0447,
          -0.9786,  1.2151, -0.4813, -1.1090,  2.2127, -1.1518, -1.4773,
           0.0672,  0.9415,  1.7102, -0.5127,  0.9629,  0.3157, -0.5821,
           0.7984, -1.2003,  0.8660,  0.8621,  0.7764,  1.4267,  1.1908,
           0.2688,  0.7186,  0.8281, -0.5400, -0.2336,  0.4387, -1.3181,
           1.3434, -0.6203, -0.4894, -0.3921, -0.5203,  1.7610,  0.4928,
           0.7221,  1.2011,  0.7298,  1.0083, -1.2

(tensor([[[ 0.3125,  1.1110,  2.5300, -0.8677, -0.6701,  0.8130,  0.8854,
           1.1803, -0.0771, -0.1987,  0.0096, -0.6858, -0.7686, -1.9430,
          -0.1218, -0.6328, -1.0675,  0.0803,  0.7564, -0.6123, -1.1549,
           0.0140, -0.3958, -0.5268,  1.3671, -0.5576, -1.0964,  0.8003,
          -1.3204,  0.6286, -1.7599,  0.2477, -0.5289, -1.1711, -1.2622,
           1.8139,  0.3481, -0.7732, -0.3589,  0.0267, -1.4591,  1.0054,
           0.4345,  0.7271, -0.8810,  1.8014, -1.0001, -0.9877, -0.0271,
           1.4306, -0.5837,  2.2067, -1.5930, -0.3819,  0.7011,  0.4766,
          -0.6624, -0.4266, -0.2487, -1.2212, -0.0708, -0.8558, -0.2767,
          -1.2946,  0.3509,  0.3346, -0.1726, -0.7191,  0.4546,  1.1010,
          -0.5663,  1.0860,  1.2272,  1.2889,  0.1249, -0.5247,  0.0659,
           1.4471, -1.0483,  0.1425, -0.3376,  0.4345,  0.8079,  0.7084,
           1.0177, -1.2358,  1.4944,  0.6666,  1.0268, -1.2400,  0.3220,
           0.2100, -2.6804,  0.1378, -0.2494, -1.3

(tensor([[[-0.3586, -0.4935, -1.4580, -1.1335, -0.3257,  2.0823,  0.7269,
           0.5325, -0.7916,  0.9221,  0.0445, -0.0122,  0.7179, -0.6552,
          -1.7834, -0.5389,  1.4039,  0.8988, -0.3924, -1.5420,  1.7954,
           0.2983, -0.5502, -0.0369,  0.1749,  1.9214,  0.6152,  0.1443,
           0.0242, -0.9093,  0.9872,  0.3064, -0.1444, -0.3438, -0.0083,
          -0.2571, -2.2128,  1.1682, -0.6639,  1.2445, -0.4691, -0.2873,
           1.1801,  1.8503, -1.4499,  0.3650,  0.9163,  1.4555, -0.7756,
          -1.6544, -0.3930,  1.2703, -0.2433, -0.5616, -0.8512, -0.6068,
           0.6352, -0.4335,  1.2754, -0.0526, -1.2957, -1.1288, -1.7648,
          -1.1737,  0.6343, -0.2827,  0.4179,  0.4315,  0.0348,  1.5854,
          -0.8746, -0.2596, -0.8995, -0.3292,  0.1836, -1.1194,  0.7863,
          -0.6631,  0.3943, -0.1094, -0.1894,  0.5176,  1.1670,  0.3920,
          -0.3182, -0.3215,  0.7961,  0.6675,  1.1107,  1.2228,  0.8447,
          -0.8697, -1.4701, -1.4047,  1.5024, -1.8

(tensor([[[ 1.7461, -0.9538, -2.0993,  0.2771, -0.4368,  0.4160, -0.7784,
           2.5428,  0.0347, -0.8368, -0.3309, -0.4224,  0.2626, -1.5379,
          -0.2974, -0.0734,  1.0527, -1.4722,  0.5334,  1.5357, -0.4240,
          -1.0638, -0.5192, -0.3662, -1.0504,  0.3955,  0.2059,  0.5675,
          -1.6570,  0.8878, -0.0671, -0.0601,  1.0376,  0.0185,  0.0168,
          -0.0570,  0.5704,  0.9012,  0.7234, -0.2515,  2.3245,  0.0684,
          -0.3608, -0.1424,  0.4611,  0.3222, -0.5703,  1.0298,  0.3972,
          -1.0366, -2.2328,  1.3398, -0.4505,  0.3441, -0.8190, -0.8747,
          -0.3358,  0.6242, -0.2776, -0.2447, -0.5648, -0.5707, -0.1914,
           1.1617, -0.3034, -1.0681,  0.8335, -0.1779, -0.3712,  0.7676,
           0.4257,  0.7248,  0.1989, -0.2306, -0.7639, -1.0081,  0.6699,
           0.8190, -1.1139,  2.0954,  0.9737,  0.0135,  0.3959, -2.4827,
          -0.7346, -0.1479, -1.3060, -0.8524, -1.4398, -1.3762, -1.9594,
          -0.6014, -0.8378, -1.1022, -0.3282,  1.1

(tensor([[[ 0.2589,  2.0427, -0.9805,  0.0768,  1.0377, -0.3949,  0.0965,
           1.2495,  0.0143, -1.9658,  1.1838, -0.8295,  1.6260, -1.2727,
           0.0900,  1.2034,  1.1159, -2.2313, -0.7006,  0.2766,  0.0861,
           1.9647,  1.7513, -0.3231, -1.9829, -1.7489,  0.0631, -1.5741,
           0.2110,  0.3345,  1.2419, -0.4784,  0.0650,  0.2307, -0.4999,
          -0.2501,  0.8538, -0.5656, -0.7875,  0.7962,  0.1640, -0.8199,
          -2.0084,  0.1070, -0.9191,  1.3288, -0.4899, -0.4054, -1.6323,
           0.3049, -2.7075, -0.6550, -0.3366, -0.3271,  1.0126,  0.5318,
          -0.0605, -0.1088,  0.8303, -0.0152,  0.1872, -1.0559,  0.9556,
          -1.0070, -0.1560,  0.3835,  0.8108,  0.3381,  0.1732, -0.9277,
          -0.9031,  0.6379, -0.8772, -1.5627,  0.0833,  0.0937,  1.0575,
           2.0299, -2.0783, -0.9930,  0.3183,  0.8008,  0.4847, -0.2082,
          -0.5498,  1.8365,  0.3115, -1.0418,  1.4138, -0.9523, -0.9287,
          -1.0380,  0.6038,  0.1228,  0.4530, -0.8

In [12]:
for wl in words_list:
    print(wl)

['ad', 'es', 'anging', 'god']
['e', 'ens', 'enging', 'eng']
['de', 'ges', 'dirng', 'ged']
['a', 'ge', 'aging', 'ged']
['ro', 'les', 'ling', 'ling']
['y', 'as', 'ying', '']
['re', 'res', 'reing', 'or']
['e', 'es', 'inging', 'an']
['', 's', 'ging', 'ged']
['te', 'tes', 'tenging', 'tod']
['o', 'o', 'og', 'o']
['', 's', 'sing', 'wed']
['', 'se', 'ding', '']
['', 'se', 'ding', 'wed']
['se', 'ses', 'inging', 'ged']
['g', 'ges', 'ging', 'ged']
['a', 'as', 'aging', 'ad']
['se', 'es', 'seng', 'sed']
['pe', 'ges', 'ging', 'ged']
['g', 'gs', 'ging', 'g']
['', 'ers', 'ering', 'wed']
['', 'es', 'ing', 'wed']
['ce', 'ers', 'cering', 'ced']
['h', 'hes', 'hoing', 'hod']
['ge', 'ges', 'gering', 'ged']
['s', 'ses', 'sing', 'thed']
['', 'se', 'sing', 'wed']
['a', 'as', 'aring', 'ad']
['ad', 'ades', 'adging', 'ad']
['', 'res', 'ricng', '']
['d', 'des', 'ding', 'd']
['', 'ges', 'gering', 'god']
['as', 'ees', 'erming', 'wed']
['', 'tes', 'ting', 'ted']
['', 's', 'inging', 'wed']
['r', 'res', 'ro', 'ro']
[''

In [127]:
def evaluate(test_data, test_cond, encoder, decoder):
    
    test_score = 0
    for i in range(len(test_data)):
        input_data_tensor = torch.tensor([ch_to_ix[ch] for ch in test_data[i][0]], device=device)
        input_cond_tensor = torch.tensor(cond_to_ix[test_cond[i][0]], device=device)
        target_cond_tensor = torch.tensor(cond_to_ix[test_cond[i][1]], device=device)
        

In [128]:
a

tensor([[1]])

In [258]:
train_loss_list_cyclical = train_loss_list.copy()

In [259]:
test_score_list_cyclical = test_score_list.copy()

In [290]:
train_loss_list_monotonic = train_loss_list.copy()
test_score_list_monotonic = test_score_list.copy()

In [179]:
b.view(-1)

tensor([1, 1])

In [264]:
for i in trange(len(test_data)):
    input_data_tensor = torch.tensor([ch_to_ix[ch] for ch in test_data[i][0]], device=device).view(-1, 1)
    input_cond_tensor = torch.tensor(cond_to_ix[test_cond[i][0]], device=device)
    target_cond_tensor = torch.tensor(cond_to_ix[test_cond[i][1]], device=device)
    pred = predict(input_data_tensor, input_cond_tensor, target_cond_tensor, encoder, decoder)
    score = compute_bleu(pred, test_data[i][1])
    print(pred, test_data[i][1])

100%|██████████| 10/10 [00:00<00:00, 314.86it/s]

abandoned abandoned
abetting abetting
begins begins
expends expends
sends sends
splitting splitting
flare flare
function function
functioned functioned
heals heals


In [266]:
torch.empty(5).normal_()

tensor([ 0.7531, -1.5099,  0.5967,  0.7548, -0.4312])

In [268]:
t = encoder.init_hidden()

In [271]:
t[0].normal_()

tensor([[[ 0.4233, -0.2107,  1.3108, -0.4431, -1.2059,  0.1561, -0.6079,
          -1.0967,  0.0221,  1.1125, -1.6064,  0.3629,  1.0480, -0.8899,
          -0.8501, -0.0663, -0.3708, -1.1808,  2.3166, -1.3944, -1.2858,
           0.5931,  0.0908, -0.0376, -0.9505, -0.4532,  1.5534, -1.1819,
           0.1057, -0.7331,  1.4506, -0.1566,  0.2780,  0.3375, -0.1828,
          -0.8213,  0.5343, -1.4857,  0.2811,  1.7749, -0.4735,  0.5542,
          -1.4555, -0.6746, -0.8415,  0.9433, -0.7942, -0.0576,  0.9452,
          -1.9283, -0.1319, -1.5095,  0.5740, -0.3545, -0.2987, -0.4119,
           1.0933, -1.3812, -1.8943, -0.0090, -0.3866, -0.5335, -0.8637,
          -0.4162,  0.2573, -0.9612,  0.5147,  0.4524,  1.3464,  0.5403,
           1.2908, -1.7049, -0.4026,  1.9600,  0.6947, -0.0949, -0.9394,
          -0.7928,  2.4211,  0.9984,  1.1858, -1.0931, -0.2186,  0.4929,
           0.8675,  1.2076,  0.5002, -0.1441, -0.1893, -0.1253, -0.0392,
           0.5997, -0.3665,  0.3795, -2.2488,  1.17

In [305]:
# Encoder & Decoder
encoder_0 = EncoderRNN(vocab_size, hidden_size, latent_size, num_conds, cond_size).to(device)
decoder_0 = DecoderRNN(latent_size+cond_size, vocab_size).to(device)

# Train
encoder_0, decoder_0, train_loss_list_0, test_score_list_0 = train(encoder_0, decoder_0,
                                                           train_data, train_cond, test_data, test_cond,
                                                           kl_anealing_func=get_kl_anealing_func('cyclical', period=10000),
                                                           teacher_forcing_ratio=0,
                                                           num_epochs=num_epochs)

  0%|          | 2/1227 [00:00<01:40, 12.25it/s]

Epoch: 1 / 20

Train Cross Entropy Loss: 2.621463323750272
Train KL Loss: 11.985321844459648

Test Cross Entropy Loss: 14.8517427444458
Test KL Loss: 23.800907135009766
Test BLEU-4 Score: 0.06651283155781636




  0%|          | 1/1227 [00:00<02:03,  9.91it/s]

Epoch: 2 / 20

Train Cross Entropy Loss: 1.6178162841551882
Train KL Loss: 214.45321799296136

Test Cross Entropy Loss: 16.460832595825195
Test KL Loss: 348.7698669433594
Test BLEU-4 Score: 0.16390657654389604




  0%|          | 2/1227 [00:00<01:55, 10.58it/s]

...

Epoch: 18 / 20

Train Cross Entropy Loss: 0.015266353100239917
Train KL Loss: 1291.9690145934544

Test Cross Entropy Loss: 0.10948039591312408
Test KL Loss: 1045.0169677734375
Test BLEU-4 Score: 1.0




  0%|          | 2/1227 [00:00<01:45, 11.61it/s]

Epoch: 19 / 20

Train Cross Entropy Loss: 0.014009266868512523
Train KL Loss: 1318.8346170717139

Test Cross Entropy Loss: 0.10826517641544342
Test KL Loss: 1026.8331298828125
Test BLEU-4 Score: 1.0




100%|██████████| 10/10 [00:00<00:00, 320.06it/s]

Epoch: 20 / 20

Train Cross Entropy Loss: 0.010027530158666104
Train KL Loss: 1340.1527442787044

Test Cross Entropy Loss: 0.10461216419935226
Test KL Loss: 1019.2639770507812
Test BLEU-4 Score: 1.0




In [306]:
# Encoder & Decoder
encoder_25 = EncoderRNN(vocab_size, hidden_size, latent_size, num_conds, cond_size).to(device)
decoder_25 = DecoderRNN(latent_size+cond_size, vocab_size).to(device)

# Train
encoder_25, decoder_25, train_loss_list_25, test_score_list_25 = train(encoder_25, decoder_25,
                                                           train_data, train_cond, test_data, test_cond,
                                                           kl_anealing_func=get_kl_anealing_func('cyclical', period=10000),
                                                           teacher_forcing_ratio=0.25,
                                                           num_epochs=num_epochs)

  0%|          | 1/1227 [00:00<02:04,  9.81it/s]

Epoch: 1 / 20

Train Cross Entropy Loss: 2.5685994831667553
Train KL Loss: 12.619815396375223

Test Cross Entropy Loss: 16.137582778930664
Test KL Loss: 33.84439468383789
Test BLEU-4 Score: 0.09928773400135933




  0%|          | 2/1227 [00:00<01:24, 14.56it/s]

Epoch: 2 / 20

Train Cross Entropy Loss: 1.5874917947810423
Train KL Loss: 483.41315448941566

Test Cross Entropy Loss: 12.941168785095215
Test KL Loss: 328.1576843261719
Test BLEU-4 Score: 0.19489470099666256




  0%|          | 2/1227 [00:00<01:51, 10.94it/s]

...

Epoch: 18 / 20

Train Cross Entropy Loss: 0.01236524084364917
Train KL Loss: 286974.92428928986

Test Cross Entropy Loss: 0.08162911981344223
Test KL Loss: 3501.729248046875
Test BLEU-4 Score: 1.0




  0%|          | 2/1227 [00:00<01:42, 11.98it/s]

Epoch: 19 / 20

Train Cross Entropy Loss: 0.010259658841725949
Train KL Loss: 330630.84126113646

Test Cross Entropy Loss: 0.09209506213665009
Test KL Loss: 3839.728271484375
Test BLEU-4 Score: 1.0




100%|██████████| 10/10 [00:00<00:00, 308.16it/s]

Epoch: 20 / 20

Train Cross Entropy Loss: 0.009653625063210615
Train KL Loss: 315045.04113369406

Test Cross Entropy Loss: 0.055579137057065964
Test KL Loss: 3017.506591796875
Test BLEU-4 Score: 1.0




In [ ]:
# Encoder & Decoder
encoder_50 = EncoderRNN(vocab_size, hidden_size, latent_size, num_conds, cond_size).to(device)
decoder_50 = DecoderRNN(latent_size+cond_size, vocab_size).to(device)

# Train
encoder_50, decoder_50, train_loss_list_50, test_score_list_50 = train(encoder_50, decoder_50,
                                                           train_data, train_cond, test_data, test_cond,
                                                           kl_anealing_func=get_kl_anealing_func('cyclical', period=10000),
                                                           teacher_forcing_ratio=0.50,
                                                           num_epochs=num_epochs)

  0%|          | 2/1227 [00:00<01:46, 11.53it/s]

Epoch: 1 / 20

Train Cross Entropy Loss: 2.4089470976754654
Train KL Loss: 21.66898718494917

Test Cross Entropy Loss: 16.570415496826172
Test KL Loss: 60.777400970458984
Test BLEU-4 Score: 0.14470810260043157




  0%|          | 2/1227 [00:00<01:33, 13.05it/s]

Epoch: 2 / 20

Train Cross Entropy Loss: 1.3119903259784897
Train KL Loss: 272.65956233908577

Test Cross Entropy Loss: 12.476791381835938
Test KL Loss: 368.546630859375
Test BLEU-4 Score: 0.2768408045530575




  0%|          | 2/1227 [00:00<01:41, 12.02it/s]

Epoch: 3 / 20

Train Cross Entropy Loss: 0.5790610207586062
Train KL Loss: 669.0296185256929

Test Cross Entropy Loss: 13.550272941589355
Test KL Loss: 513.8779296875
Test BLEU-4 Score: 0.4620740188731863




  0%|          | 1/1227 [00:00<02:04,  9.87it/s]

Epoch: 4 / 20

Train Cross Entropy Loss: 0.34994948927768
Train KL Loss: 1251.3374488413965

Test Cross Entropy Loss: 13.2019624710083
Test KL Loss: 567.2276611328125
Test BLEU-4 Score: 0.41598079182848435




  5%|▌         | 66/1227 [00:05<01:38, 11.74it/s]

In [ ]:
# Encoder & Decoder
encoder_75 = EncoderRNN(vocab_size, hidden_size, latent_size, num_conds, cond_size).to(device)
decoder_75 = DecoderRNN(latent_size+cond_size, vocab_size).to(device)

# Train
encoder_75, decoder_75, train_loss_list_75, test_score_list_75 = train(encoder_75, decoder_75,
                                                           train_data, train_cond, test_data, test_cond,
                                                           kl_anealing_func=get_kl_anealing_func('cyclical', period=10000),
                                                           teacher_forcing_ratio=0.75,
                                                           num_epochs=num_epochs)

In [ ]:
# Encoder & Decoder
encoder_100 = EncoderRNN(vocab_size, hidden_size, latent_size, num_conds, cond_size).to(device)
decoder_100 = DecoderRNN(latent_size+cond_size, vocab_size).to(device)

# Train
encoder_100, decoder_100, train_loss_list_100, test_score_list_100 = train(encoder_100, decoder_100,
                                                           train_data, train_cond, test_data, test_cond,
                                                           kl_anealing_func=get_kl_anealing_func('cyclical', period=10000),
                                                           teacher_forcing_ratio=1,
                                                           num_epochs=num_epochs)

In [14]:
Gaussian_score(words_list)

0.0

In [16]:
plt.plot(test_stest_score_list_0core_list_0)

NameError: name 'test_score_list_0' is not defined

In [17]:
test_score_list_0

NameError: name 'test_score_list_0' is not defined

In [13]:
def generate_words(encoder, decoder, num=100):
    
    encoder.eval()
    decoder.eval()
    
    cond_size = encoder.cond_size
    embedded_conditions = {}
    for cond in conditions:
        cond_tensor = torch.tensor(cond_to_ix[cond], device=device)
        embedded_conditions[cond] = encoder.condition_embedding(cond_tensor)
    
    words_list = []
    with torch.no_grad():
        for _ in range(num):
            noise = decoder.init_hidden()

            # Generate Gaussian noise
            noise = (noise[0].normal_(std=0.7), noise[1].normal_(std=0.7))

            # Generate words with 4 different tenses
            words = []
            for embedded_cond in embedded_conditions.values():
                
                noise[0][:, :, -cond_size:] = embedded_cond
                noise[1][:, :, -cond_size:] = embedded_cond
                
                decoder_input = torch.tensor([[SOS_token]], device=device)
                decoder_hidden = noise
            
                word = ''
                while True:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()
                    
                    if decoder_input == EOS_token:
                        break
                        
                    word += ix_to_ch[decoder_input.item()]
                words.append(word)
            words_list.append(words)
    return words_list

In [14]:
# Define hyperparameters
hidden_size = 512
latent_size = 64
vocab_size = 28
num_conds = 4
cond_size = 32
teacher_forcing_ratio = 0.5
learning_rate = 0.005
MAX_LENGTH = 40
num_epochs = 20

# Encoder & Decoder
encoder = EncoderRNN(vocab_size, hidden_size, latent_size, num_conds, cond_size).to(device)
decoder = DecoderRNN(latent_size+cond_size, vocab_size).to(device)
E

# Evaluate


# Generate
words_list = generate_words(encoder, decoder, num=100)
for words in words_list:
    print(words)
print(Gaussian_score(words_list))

['u', 'ses', 'inding', 'aded']
['y', 'ses', 'ying', 'od']
['e', 'es', 'necing', 'ed']
['', 'es', 'iding', 'ked']
['y', 'ys', 'ying', 'ad']
['a', 'ds', 'zing', 'ad']
['de', 'ses', 'ding', 'ded']
['y', 'eds', 'ying', 'oded']
['fe', 'fes', 'ying', 'fed']
['e', 'es', 'ending', 'd']
['y', 'es', 'eading', 'ad']
['e', 'es', 'eying', 'ed']
['e', 'es', 'ling', 'ad']
['id', 'eds', 'eding', 'ud']
['e', 'eds', 'eding', 'ed']
['u', 'ies', 'ying', 'ud']
['ee', 'eds', 'eying', 'ad']
['y', 'es', 'eying', 'wed']
['ud', 'ses', 'ging', 'ad']
['ue', 'eds', 'eding', 'ued']
['e', 'es', 'eeing', 'ad']
['w', 'ses', 'wing', 'wed']
['y', 'eds', 'eaing', 'ded']
['e', 'es', 'ezing', 'oud']
['', 'ses', 'ying', 'wed']
['d', 'des', 'ding', 'wed']
['uy', 'is', 'ing', 'ung']
['o', 'es', 'eding', 'ad']
['ie', 'es', 'inging', 'od']
['ue', 'ses', 'neing', 'wed']
['e', 'es', 'eving', 'ad']
['ed', 'des', 'deing', 'od']
['', 's', 'sing', 'de']
['e', 'es', 'ening', 'ed']
['u', 'eds', 'ling', 'ad']
['y', 'ges', 'yeding', 'wed